In [2]:
import gym
import neat
import gym
import numpy as np
import time
from abc import ABC, abstractmethod

In [10]:
render_slowdown = 0.03

def run(config_file, eval_genomes, visual_reporter):
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    p.add_reporter(neat.StatisticsReporter())
    p.add_reporter(neat.Checkpointer(5))
    p.add_reporter(visual_reporter)

    # Run for up to 300 generations.
    winner = p.run(eval_genomes, 500)
    return winner

def simulate(actor, turns, render):
    env = gym.make(simulation_type)
    observation = env.reset()
    
    for i in range(turns):
        action = actor.act(observation)
        observation, fitness, done, info = env.step(action)
        actor.learn(observation, fitness)
        
        if render == True:
            time.sleep(render_slowdown)
            env.render()
            
        if done:
            break
    
    env.close()

def get_best_genome(genomes):
    best_genome = None
    max_fitness = -1000
    for genome_id, genome in genomes:
        if max_fitness < genome.fitness:
            max_fitness = genome.fitness
            best_genome = genome
    
    return best_genome

class Actor(ABC):
    def __init__(self, genome, net, isLearning):
        self.genome = genome
        self.net = net
        self.isLearning = isLearning
            
    def learn(self, observation, fitness):
        if self.isLearning == True:
            self.genome.fitness = self.evaluate(observation, fitness)
            
    @abstractmethod
    def act(self, observation):
        pass
    
    @staticmethod
    @abstractmethod
    def evaluate(observation, fitness):
        return fitness
    
class VisualReporter(neat.reporting.BaseReporter):
    def __init__(self, actor, min_generation, generation_gap):
        neat.reporting.BaseReporter.__init__(self)
        self.actor = actor
        self.current_generation = 0
        self.min_generation = min_generation
        self.generation_gap = generation_gap
        
    def end_generation(self, config, population, species_set):
        self.current_generation += 1
        
    def post_evaluate(self, config, population, species, best_genome):
        if self.current_generation > self.min_generation and self.current_generation % self.generation_gap == 0:
            best_net = neat.nn.FeedForwardNetwork.create(best_genome, config)
            best_acrobot = self.actor(best_genome, best_net, False)
            simulate(best_acrobot, num_turns, True)

In [7]:
num_turns = 200
render_generation = 1
min_render_generation = 0
simulation_type = "Acrobot-v1"

class Acrobot(Actor):
    def act(self, observation):
        output = self.net.activate(observation)[0]

        if output > 0.75:
            return 1
        elif output <0.25:
            return -1
        else:
            return 0
    
    @staticmethod
    def evaluate(observation, fitness):
        return fitness
        
def eval_genomes(genomes, config):
    global cur_generation
    
    for genome_id, genome in genomes:
        genome.fitness = -1000
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        acrobot = Acrobot(genome, net, True)
        simulate(acrobot, num_turns, False)
                
    best_genome = get_best_genome(genomes)

visual_reporter = VisualReporter(Acrobot, min_render_generation, render_generation)
best = run('neat_acrobot.cfg', eval_genomes, visual_reporter)


 ****** Running generation 0 ****** 

Population's average fitness: -0.88500 stdev: 0.31902
Best fitness: 0.00000 - size: (1, 6) - species 1 - id 15
0
0
1

Best individual in generation 0 meets fitness threshold - complexity: (1, 6)


In [4]:
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     'neat_acrobot.cfg')

best_net = neat.nn.FeedForwardNetwork.create(best, config)
best_acrobot = Acrobot(best, best_net, False)
simulate(best_acrobot, num_turns, True)

In [8]:
num_turns = 500
render_generation = 10
min_render_generation = 20
simulation_type = "MountainCarContinuous-v0"

class MountainCar(Actor):        
    def act(self, observation):
        output = self.net.activate(observation)
        return ([output[0] * 2 - 1])
    
    @staticmethod
    def evaluate(observation, fitness):
        return fitness + 100 * abs(observation[0])
        

def eval_genomes(genomes, config):
    global cur_generation
    
    for genome_id, genome in genomes:
        genome.fitness = 0
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        mountain_car = MountainCar(genome, net, True)
        simulate(mountain_car, num_turns, False)
                
    best_genome = get_best_genome(genomes)


visual_reporter = VisualReporter(MountainCar, min_render_generation, render_generation)
best = run('neat_mountain.cfg', eval_genomes, visual_reporter)


 ****** Running generation 0 ****** 

Population's average fitness: 52.10144 stdev: 25.19973
Best fitness: 108.73667 - size: (1, 2) - species 1 - id 1
0
20
10
Average adjusted fitness: 0.432
Mean genetic distance 1.870, standard deviation 0.669
Population of 200 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   136    108.7    0.503     0
     2    0    64    106.0    0.361     0
Total extinctions: 0
Generation time: 1.294 sec

 ****** Running generation 1 ****** 

Population's average fitness: 64.89586 stdev: 25.38917
Best fitness: 146.12687 - size: (1, 1) - species 1 - id 355
0
20
10
Average adjusted fitness: 0.419
Mean genetic distance 1.684, standard deviation 0.614
Population of 200 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1   107    146.1    0.481     0
     2    1    93    102.4    0.357     1
Total extinctions: 0
Generation time: 1.25

KeyboardInterrupt: 

In [17]:
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     'neat_mountain.cfg')

best_net = neat.nn.FeedForwardNetwork.create(best, config)
best_mountain_car = MountainCar(best, best_net, False)
simulate(best_mountain_car, num_turns, True)

In [11]:
num_turns = 250
render_generation = 3
min_render_generation = 4
simulation_type = "LunarLanderContinuous-v2"

class LunarLander(Actor):
    def act(self, observation):
        output = self.net.activate(observation)
        return ([i*2-1 for i in output])
    
    @staticmethod
    def evaluate(observation, fitness):
        return fitness - 10 * observation[1] - abs(observation[0]) - observation[2]

def eval_genomes(genomes, config):
    global cur_generation
    
    for genome_id, genome in genomes:
        genome.fitness = 0
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        lunar_lander = LunarLander(genome, net, True)
        simulate(lunar_lander, num_turns, False)
                
    best_genome = get_best_genome(genomes)

visual_reporter = VisualReporter(LunarLander, min_render_generation, render_generation)
best = run('neat_lander.cfg', eval_genomes, visual_reporter)


 ****** Running generation 0 ****** 



/Users/maksimkarabashev/opt/anaconda3/envs/Jupyter/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Population's average fitness: -106.30062 stdev: 10.92886
Best fitness: -98.17593 - size: (2, 16) - species 1 - id 86
Average adjusted fitness: 0.864
Mean genetic distance 1.355, standard deviation 0.333
Population of 100 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   100    -98.2    0.864     0
Total extinctions: 0
Generation time: 3.200 sec

 ****** Running generation 1 ****** 

Population's average fitness: -105.46797 stdev: 12.23360
Best fitness: -54.52166 - size: (2, 16) - species 1 - id 175
Average adjusted fitness: 0.571
Mean genetic distance 1.427, standard deviation 0.316
Population of 100 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1   100    -54.5    0.571     0
Total extinctions: 0
Generation time: 2.910 sec (3.055 average)

 ****** Running generation 2 ****** 

Population's average fitness: -103.81532 stdev: 8.26643
Best fitness: -

In [16]:
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     'neat_lander.cfg')

best_net = neat.nn.FeedForwardNetwork.create(best, config)
best_lunar_lander = LunarLander(best, best_net, False)
simulate(best_lunar_lander, num_turns, True)